# <center>**Pricing Options with PDE Implicit Scheme**</center>

This project presents pricing of the call-spread and put options with a use of partial differential equation (PDE) implicit scheme. Obtained prices are compared with the prices given by Black-Scholes formula.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
#import matplotlib.animation as animation
from mpl_toolkits import mplot3d
plt.rcParams["figure.figsize"] = (16,9)

## Call-spread

Fisrt we use the Black-Scholes model to price the call-spread. Its price is equal to the difference between the price of the long leg and short leg. It's due to the fact that buying a call-spread with trike prices K1 and K2 (with K1 < K2) is equivalent to buying a call with strike price K1 and selling a call with strike price K2.

In [9]:
def BS_formula_call(S_0, K, r, sigma, T):
    d_1 = (np.log(S_0 / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d_2 = d_1 - sigma * np.sqrt(T)
    return S_0 * norm.cdf(d_1) - K * np.exp(-r * T) * norm.cdf(d_2)

In [10]:
S_0 = 10.
K1 = 10.
K2 = 15.
r = 0.02
sigma = 0.25
T = 3.
g = lambda S: np.maximum(S - K1, 0.) - np.maximum(S - K2, 0.)

In [11]:
BS_formula_call(S_0, K1, r, sigma, T) - BS_formula_call(S_0, K2, r, sigma, T)

1.3585071391775059

Now we develop the function to solve the PDE implicit scheme for the same call-spread. 
We set the following values at the boundaries:
- payoff at maturity (right boundary): max(S - K1, 0) - max(S - K2, 0); the difference of the payoffs of two calls (as explained earlier)
- price of the option at the very low asset's spot price level (lower boundary): 0; we assume that the probability of the spot rising up to the strike is so low we can set it as 0, therefore, the probability that the option will be worth exercising is 0 
- price of the option at the very high asset's spot price level (upper boundary): (K2 - K1) * exp(-rt); we assume that the probability of the spot falling down below the strikes is so low we can set it as 0, therefore, the price of the option is the discounted value of it's expected (in the risk-neutral probability space) payoff at the maturity, which is, at this very high spot level, equal to the discounted value of K2 - K1

We also have to choose upper and lower bounds for the spot price. We set upper bound as 4 times the initial spot and lower bound as 1/4 of the initial spot, as for these levels the possibility of falling (or rising) appropriately much is so low that this possibility makes almost no impact on the option price and thus, it can be ommited.
The rest is the same as for the plain vanilla call PDE implicit scheme.

In [12]:
def pde_scheme_implicit_call_spread(S_0, r, sigma, T, K1, K2, H, nb_x_side, nb_t):
    nb_x = 2 * nb_x_side + 1
    xs = np.linspace(np.log(S_0) - H, np.log(S_0) + H, nb_x)
    dx = 2. * H / (nb_x - 1.)
    dt = T / (nb_t - 1.)
    ts = np.linspace(0, T, nb_t)
    p = np.empty([nb_x, nb_t])
    g = lambda S: np.maximum(S - K1, 0.) - np.maximum(S - K2, 0.)
    p[:, 0] = g(np.exp(xs))
    p[0, :] = 0.
    p[-1, :] = K2 * np.exp(-r * ts) - K1 * np.exp(-r * ts)
    d = 1. + dt * (r + (r - 0.5 * sigma ** 2) / dx + sigma ** 2 / dx ** 2)
    #print(d)
    sup_d = -dt * ((r - 0.5 * sigma ** 2) / dx + 0.5 * sigma ** 2 / dx ** 2)
    inf_d = -dt * (0.5 * sigma ** 2 / dx ** 2)
    A = np.diag(d * np.ones(nb_x - 2)) + np.diag(sup_d * np.ones(nb_x - 3), 1) + np.diag(inf_d * np.ones(nb_x - 3), -1)
    v = np.zeros_like(p[1:-1, 0])
    v[-1] = 1.
    invA = np.linalg.inv(A)
    for t in range(1, nb_t):
        p[1:-1, t] = invA @ (p[1:-1, t - 1] - sup_d * p[-1, t] * v)
    return p, p[nb_x_side, -1]

In [13]:
H = np.log(40) - np.log(10)
nb_x_side = 200
nb_t = 25000

In [14]:
table, price = pde_scheme_implicit_call_spread(S_0, r, sigma, T, K1, K2, H, nb_x_side, nb_t)
price

1.357907469753652

The obtained price is close enough to the price obtained from the Black-Scholes model.

## Put with maturity 10 years

Fisrt we use the Black-Scholes model to price the put.

In [15]:
S_0=10.
K=8.
r=0.02
sigma=0.25
T=10. 
g=lambda S: np.maximum(K-S,0.)

In [16]:
# here BS formula for a put
from scipy.stats import norm
def BS_formula_put(S_0,K,r,sigma,T):
    d_1 = (np.log(S_0/K) + (r + sigma**2/2)*T)/(sigma * np.sqrt(T))
    d_2 = d_1 - sigma * np.sqrt(T)
    return -S_0 * norm.cdf(-d_1) + K * np.exp(-r*T) * norm.cdf(-d_2)

In [17]:
BS_formula_put(S_0,K,r,sigma,T)

1.149911346784067

Now we develop the function to solve the PDE implicit scheme for the same put. 
We set the following values at the boundaries:
- payoff at maturity (right boundary): max(K - S, 0)
- price of the option at the very high asset's spot price level (upper boundary): 0; we assume that the probability of the spot falling down to the strike is so low we can set it as 0, therefore, the probability that the option will be worth exercising is 0 
- price of the option at the very low asset's spot price level (lower boundary): K * exp(-rt) - S; we assume that the probability of the spot rising up to the strike is so low we can set it as 0, therefore, the price of the option is the discounted value of it's expected (in the risk-neutral probability space) payoff at the maturity, which is, at this very high spot level, equal to the discounted value of K - E[S]

We also have to "rotate" the vector v upside-down, since for a put the values at the lower boundry are not equal to 0 (and equal to 0 at the upper), as opposed to the call.

We also have to choose upper and lower bounds for the spot price. We set upper bound as 4 times the initial spot and lower bound as 1/4 of the initial spot, since for these levels the possibility of falling (or rising) appropriately much is so low that this possibility makes almost no impact on the option price and thus, it can be ommited.
The rest is the same as for the plain vanilla call PDE implicit scheme.

In [18]:
#implicit scheme
def pde_scheme_implicit_put(S_0,r,sigma,T,K,H,nb_x_side,nb_t):
    nb_x = 2 * nb_x_side+1
    xs = np.linspace(np.log(S_0)-H, np.log(S_0)+H, nb_x)
    dx = 2.*H/(nb_x - 1.)
    dt = T/(nb_t - 1.)
    ts = np.linspace(0,T,nb_t)
    p = np.empty([nb_x,nb_t])
    g = lambda S : np.maximum(K-S,0.)
    p[:,0] = g(np.exp(xs))
    p[0,:] = - np.exp(np.log(S_0)-H) + K * np.exp(-r*ts)
    p[-1,:] = 0.
    d = 1. + dt * (r + (r-0.5*sigma**2)/dx + sigma**2 / dx**2)
    sup_d = -dt * ((r-0.5*sigma**2)/dx + 0.5 * sigma**2 / dx**2)
    inf_d = -dt * (0.5 * sigma**2 / dx**2)
    A = np.diag(d * np.ones(nb_x-2)) + np.diag(sup_d * np.ones(nb_x-3),1) + np.diag(inf_d * np.ones(nb_x-3),-1)
    v = np.zeros_like(p[1:-1,0])
    v[0]=1.
    invA = np.linalg.inv(A)
    for t in range(1,nb_t):
        p[1:-1,t]=invA @ (p[1:-1,t-1]-sup_d * p[0,t] * v)
    return p,p[nb_x_side,-1]

In [19]:
H = np.log(40) - np.log(10)
nb_x_side = 200
nb_t = 25000

In [20]:
table,price = pde_scheme_implicit_put(S_0, r, sigma, T, K, H, nb_x_side, nb_t)
price

1.1470538283440983

The obtained price is close enough to the price obtained from the Black-Scholes model.